In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import ast
import json

In [12]:

# Load your data (replace with your file path)
#data= pd.read_parquet('your_landmark_data.parquet')

df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")


In [16]:
display(df_from_parquet["Keypoints"][0].shape)

(75,)

In [75]:
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}
# Step 3: Map the Label to Label_id in the DataFrame
df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
label_info = [{label, label_id} for label, label_id in label_id_mapping.items()]

In [79]:
unique_labels

array(['are you free today', 'bring water for me', 'can i help you',
       'can you repeat that please', 'comb your hair', 'congratulations',
       'could you please talk slower', 'do me a favour',
       'do not abuse him', 'do not be stubborn', 'do not hurt me',
       'do not make me angry', 'do not take it to the heart',
       'do not worry', 'do you need something', 'go and sleep',
       'had your food', 'he came by train', 'He is going into the room',
       'he is on the way', 'he she is my friend',
       'he would be coming today', 'help me', 'hi how are you',
       'how are things', 'how can i help you', 'how can i trust you',
       'how dare you', 'how old are you', 'i am (age)',
       'i am afraid of that', 'i am crying', 'i am feeling bored',
       'i am feeling cold', 'i am fine. thank you sir', 'i am hungry',
       'i am in dilemma what to do', 'i am not really sure',
       'i am really grateful', 'i am sitting in the class',
       'i am so sorry to hear that'

In [18]:
df_1=df_from_parquet[["Label_id","Sample_Number","Frame_Number","Keypoints"]]

In [19]:
df_1

,Label_id,Sample_Number,Frame_Number,Keypoints
0,0,0,1.0,"[{0.47676751017570496, 0.3017379641532898, -0...."
1,0,0,2.0,"[{0.47681325674057007, 0.30156728625297546, -0..."
2,0,0,3.0,"[{0.4772616922855377, 0.3015567362308502, -0.6..."
3,0,0,4.0,"[{0.477945476770401, 0.30148565769195557, -0.6..."
4,0,0,5.0,"[{0.47859013080596924, 0.30141815543174744, -0..."
...,...,...,...,...
51558,99,4,114.0,"[{0.49276989698410034, 0.35694217681884766, -0..."
51559,99,4,115.0,"[{0.4929289221763611, 0.3573005497455597, -0.6..."
51560,99,4,116.0,"[{0.49306631088256836, 0.35789814591407776, -0..."
51561,99,4,117.0,"[{0.49327489733695984, 0.3583262860774994, -0...."


In [37]:
df_1["Label_id"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int64)

In [43]:
df = pd.DataFrame(df_1)
display(df["Keypoints"][0])

array(['{0.47676751017570496, 0.3017379641532898, -0.6963335275650024}',
       '{0.489931583404541, 0.2655402421951294, -0.6616351008415222}',
       '{0.4974368214607239, 0.26715803146362305, -0.6617963910102844}',
       '{0.5050826668739319, 0.2687525749206543, -0.6618160009384155}',
       '{0.46471312642097473, 0.2656858563423157, -0.6637672781944275}',
       '{0.4561752378940582, 0.26697224378585815, -0.6639881730079651}',
       '{0.4481793940067291, 0.26809918880462646, -0.6643333435058594}',
       '{0.5169976949691772, 0.29265040159225464, -0.42892986536026}',
       '{0.43634259700775146, 0.29160308837890625, -0.4334210157394409}',
       '{0.4924103319644928, 0.3457692265510559, -0.6073712706565857}',
       '{0.4615137279033661, 0.3445237874984741, -0.6089097261428833}',
       '{0.5903962850570679, 0.5426028370857239, -0.27551522850990295}',
       '{0.36381176114082336, 0.5239225625991821, -0.26729124784469604}',
       '{0.6097986698150635, 0.8267006874084473, -0.2548

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(df_1)

# Step 1: Get unique values
unique_label_ids = df['Label_id'].unique()
unique_sample_numbers = df['Sample_Number'].unique()
unique_frame_numbers = df['Frame_Number'].unique()

num_label_ids = len(unique_label_ids)
num_sample_numbers = len(unique_sample_numbers)
num_frame_numbers = len(unique_frame_numbers)

# Assuming each keypoint array has the same length, say 10
num_keypoints = max(df['Keypoints'].apply(len))  # Adjust based on your keypoint data
display(num_keypoints)

# Step 2: Initialize the 4D array
array_shape = (num_label_ids, num_sample_numbers, num_frame_numbers, num_keypoints)
keypoints_array = np.zeros(array_shape)

# Step 3: Populate the array
for _, row in df.iterrows():
    label_idx = np.where(unique_label_ids == row['Label_id'])[0][0]
    display(label_idx)
    sample_idx = np.where(unique_sample_numbers == row['Sample_Number'])[0][0]
    display(sample_idx)
    frame_idx = np.where(unique_frame_numbers == row['Frame_Number'])[0][0]
    display(frame_idx)
    
    # Fill the corresponding position in the array
    keypoints_array[label_idx, sample_idx, frame_idx, :] = row["Keypoints"]

# Resulting 4D array
print("Shape of the keypoints array:", keypoints_array.shape)
print("Keypoints array:", keypoints_array)


In [63]:
df = pd.DataFrame(df_1)

In [64]:
df.groupby(['Label_id', 'Sample_Number'])

In [94]:
df_1.groupby(['Label_id', 'Sample_Number']).count()

Frame_Number  Keypoints
Label_id Sample_Number                         
0        0                        78         78
         1                        98         98
         2                        82         82
         3                       117        117
         4                       101        101
...                              ...        ...
99       0                       118        118
         1                       155        155
         2                       111        111
         3                       115        115
         4                       118        118

[487 rows x 2 columns]

In [101]:
df_1['Frame_Number'].unique()

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame creation (replace this with your actual DataFrame)
# Assuming df has columns: label_id, sample_number, frame_number, keypoints
# df = pd.read_csv('your_file.csv')  # Load your DataFrame

# Step 1: Group by label_id and sample_number
grouped = df_1.groupby(['Label_id', 'Sample_Number'])

# Step 2: Create a list to hold the sequences
sequences = []

# Define the number of timesteps (n)
unique_frame_numbers = df_1['Frame_Number'].unique()
n_timesteps = len(unique_frame_numbers)  # Adjust based on your requirement

for (label_id, sample_number), group in grouped:
    print("label_id",label_id)
    print("sample_number",sample_number)

    keypoints = group['Keypoints'].to_list()  # Extract the keypoints as a list

    # Create sequences of the last n_timesteps
    for i in range(len(keypoints) - n_timesteps + 1):
        sequence = keypoints[i:i + n_timesteps]
        sequences.append((label_id, sample_number, sequence))

# # Step 3: Convert to numpy array
# X = np.array([seq[2] for seq in sequences])  # Extract sequences
# y = np.array([seq[0] for seq in sequences])  # Extract labels (label_id)

# # Reshape X for LSTM (samples, timesteps, features)
# X = X.reshape((X.shape[0], n_timesteps, 75))  # Assuming keypoints are of shape (75,)

# # Now X is ready for LSTM model input
# print(f'X shape: {X.shape}, y shape: {y.shape}')


In [105]:
sequences.shape()

AttributeError: 'list' object has no attribute 'shape'

In [104]:
display(sequences)

[(36,
  2,
  [array(['{0.48430490493774414, 0.2692376375198364, -0.55692058801651}',
          '{0.5034592747688293, 0.23882728815078735, -0.5212177038192749}',
          '{0.5116249322891235, 0.24179935455322266, -0.5213238596916199}',
          '{0.5198902487754822, 0.24581296741962433, -0.5215718150138855}',
          '{0.47330009937286377, 0.2334877848625183, -0.5160020589828491}',
          '{0.4639800190925598, 0.23305445909500122, -0.5161699652671814}',
          '{0.4569339156150818, 0.23309797048568726, -0.5163710117340088}',
          '{0.5355096459388733, 0.27318257093429565, -0.3126915991306305}',
          '{0.44926613569259644, 0.25588303804397583, -0.28995421528816223}',
          '{0.5019679069519043, 0.3227653503417969, -0.4795459508895874}',
          '{0.46568986773490906, 0.31810158491134644, -0.47366347908973694}',
          '{0.5995346307754517, 0.5004698038101196, -0.1938883513212204}',
          '{0.37775418162345886, 0.48386308550834656, -0.18521994352340698}',

In [96]:
display(y)

array([36, 89], dtype=int64)

# Build and Train LSTM Neural Network

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import os

In [72]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [81]:
unique_labels.shape[0]

100

In [82]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(192,75)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(unique_labels.shape[0], activation='softmax'))

In [83]:
res = [.7, 0.2, 0.1]

In [84]:
unique_labels[np.argmax(res)]

'are you free today'

In [85]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [87]:
model.fit(X, y, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000


ValueError: in user code:

    File "C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\ranjan.patra\AppData\Roami

In [ ]:
label_info

In [47]:

# Step 1: Ensure Keypoints are converted to lists if they are not already
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Step 2: Group by the necessary columns and aggregate
grouped = df_from_parquet.groupby(['Label','Label_id', 'sample_type', 'Sample_Number']).apply(
    lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
).reset_index(name='frames_keypoints')

# Step 3: Convert the grouped result to a JSON-like structure
grouped_json = grouped.to_dict(orient='records')

# Step 5: Convert the result to a JSON string
json_output = json.dumps(grouped_json, indent=4)


In [ ]:
# Step 5: Ensure grouped_json is treated as an array
json_output_array = [grouped_json]  # or simply treat it as is
json_output = json.dumps(json_output_array, indent=4)

# If grouped_json is already a list, you can omit wrapping it:
json_output = json.dumps(grouped_json, indent=4)

# Printing or using the json output
print(json_output)

In [ ]:
# Step to remove names and keep only numbers in the grouped_json structure
def remove_names_and_keep_numbers(entry):
    return {
        'Label_id': entry['Label_id'],
        'Sample_Number': entry['Sample_Number'],
        'frames_keypoints': [
            {'Frame_Number': frame['Frame_Number'], 'Keypoints': frame['Keypoints']}
            for frame in entry['frames_keypoints']
        ]
    }

# Applying this function to the grouped_json
numeric_grouped_json = [remove_names_and_keep_numbers(item) for item in grouped_json]

# Convert to JSON string
json_output = json.dumps(numeric_grouped_json, indent=4)

# Printing or using the output
print(json_output)


In [ ]:
print(grouped_json[0])


In [ ]:
label_id_mapping

In [ ]:
data[0]

In [5]:
data = json.loads(json_output)
#print(data)

In [ ]:
np.array(array_output).shape

In [4]:
import pandas as pd
import json

# Read from a parquet file

# Proceed with mapping and processing as before
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}

df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Ensure this is a DataFrame before passing it
if isinstance(df_from_parquet, pd.DataFrame):
    grouped = df_from_parquet.groupby(['Label', 'Label_id', 'sample_type', 'Sample_Number']).apply(
        lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
    ).reset_index(name='frames_keypoints')
    
    grouped_json = grouped.to_dict(orient='records')
    json_output = json.dumps(grouped_json, indent=4)

    # Optionally save to a file
    with open('output.json', 'w') as json_file:
        json_file.write(json_output)
else:
    print("Data is not a DataFrame.")


In [ ]:
data = json.loads(json_output)
print(data[0])

In [25]:
import re
import numpy as np

def extract_keypoints(df):
    keypoints = []
    labels = []
    
    for _, row in df.iterrows():
        keypoint_str = row[1]['frames_keypoints'][0]['Keypoints']
        
        if isinstance(keypoint_str, str):
            print("Is Str")
            # Remove curly braces and split the string by commas
            keypoint_str = keypoint_str.replace('{', '').replace('}', '')
            try:
                print("Is not Str")
                # Convert the string into a list of floats
                keypoint_list = [float(val) for val in keypoint_str.split(',')]
                keypoint_str = keypoint_list.replace('{', '').replace('}', '')
            except ValueError as e:
                print(f"Error converting keypoints: {keypoint_str} -> {e}")
                continue  # Skip problematic rows if conversion fails
        else:
            keypoint_list = keypoint_str  # If not a string, assume it's already correct
        
        keypoints.append(keypoint_list)
        labels.append(row['Label'])  # Assuming label is numeric
    
    return np.array(keypoints), np.array(labels)


In [ ]:
data.iterrows()

In [ ]:
data[0]
#['frames_keypoints'][0]['Keypoints']

In [ ]:
X, y = extract_keypoints(data)


In [ ]:

# Check keypoints data and ensure it's numeric
print("Keypoints shape: ", X.shape)
print("Sample keypoints data: ", X[0])



In [ ]:
X = X.astype(np.float32)  # Convert keypoints to float32
X = X / np.max(X)  # Normalize between 0 and 1
